In [0]:
#1)- Importing key modules

In [0]:
# Let's be rebels and ignore warnings for now
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [0]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
import sys

In [3]:
! pip install turicreate

    100% |████████████████████████████████| 87.4MB 327kB/s 
    100% |████████████████████████████████| 23.8MB 1.6MB/s 
    100% |████████████████████████████████| 2.7MB 10.8MB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [0]:
import turicreate as tc

In [0]:
# 2)-Loading Data

In [0]:
url = 'https://raw.githubusercontent.com/HassanSherwani/Product_Purchase_Frequency/master/20190207_transactions%20.json'

In [0]:
transactions = pd.read_json(url, lines= True)

In [0]:
# 3)-Exploring dataset

In [10]:
transactions.head()

,id,products
0,0,"[185, 30, 77, 188, 78, 125, 45, 155, 241, 229,..."
1,1,"[119, 148, 108, 34, 157, 82, 113, 45, 165]"
2,2,"[173, 103, 229, 240]"
3,3,[91]
4,4,"[175, 192, 54, 172]"


In [17]:
transactions.shape

(2500, 2)

In [13]:
transactions.info() # checking missing values

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2500 entries, 0 to 2499
Data columns (total 2 columns):
id          2500 non-null int64
products    2500 non-null object
dtypes: int64(1), object(1)
memory usage: 58.6+ KB


In [16]:
transactions.describe()

,id
count,2500.00000
mean,1249.50000
std,721.83216
min,0.00000
25%,624.75000
50%,1249.50000
75%,1874.25000
max,2499.00000


In [0]:
### Adding Features: Create a separate dataframe for recommending users
customers=transactions['id']

In [0]:
cust_2_rec=customers[:1000]

In [21]:
### break down each list of items in the products column into rows and count the number of products bought by a user

pd.melt(transactions.head(2).set_index('id')['products'].apply(pd.Series).reset_index(), 
             id_vars=['id'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['id', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})

,id,productId,purchase_count
0,0,30.0,1
1,0,45.0,1
2,0,77.0,1
3,0,78.0,1
4,0,89.0,2
5,0,125.0,1
6,0,133.0,1
7,0,155.0,1
8,0,161.0,1
9,0,185.0,1


In [0]:
# 4)- Making data for user, item, and target field

In [22]:
### 4a)-One for purchase count
s=time.time()

data = pd.melt(transactions.set_index('id')['products'].apply(pd.Series).reset_index(), 
             id_vars=['id'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['id', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})
data['productId'] = data['productId'].astype(np.int64)

print("Execution time:", round((time.time()-s)/60,2), "minutes")

Execution time: 0.01 minutes


In [23]:
data.shape

(24811, 3)

In [24]:
data.head()

,id,productId,purchase_count
0,0,30,1
1,0,45,1
2,0,77,1
3,0,78,1
4,0,89,2


In [0]:
### 4-b)-Dummy as target 
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy

In [0]:
data_dummy = create_data_dummy(data)

In [67]:
data_dummy.head()

,id,productId,purchase_count,purchase_dummy
0,0,30,1,1
1,0,45,1,1
2,0,77,1,1
3,0,78,1,1
4,0,89,2,1


In [0]:
### 4-c)-Normalize item
"""
we normalize purchase frequency of each item across users by first creating a user-item matrix 

"""
df_matrix = pd.pivot_table(data, values='purchase_count', index='id', columns='productId')

In [29]:
df_matrix.head()

productId,1,2,3,4,5,6,7,8,9,10,...,241,242,243,244,245,246,247,248,249,250
id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
df_matrix.shape

(2378, 250)

In [31]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())

df_matrix_norm.head()

(2378, 250)


productId,1,2,3,4,5,6,7,8,9,10,...,241,242,243,244,245,246,247,248,249,250
id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
print(df_matrix_norm.shape)

(2378, 250)


In [0]:
### 4-d)- create a table for input to the modeling

d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['id'], value_name='scaled_purchase_freq').dropna()

In [35]:
data_norm.head()

,id,productId,scaled_purchase_freq
11,12,1,0.0
38,41,1,0.0
40,43,1,0.0
93,96,1,0.0
100,103,1,0.0


In [36]:
data_norm.shape

(22530, 3)

In [0]:
###4-e) a function for normalizing data
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='id', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['id'], value_name='scaled_purchase_freq').dropna()

In [0]:
# 5)-Split train and test set

In [38]:
train, test = train_test_split(data, test_size = .2)
print(train.shape, test.shape)

(19848, 3) (4963, 3)


In [0]:
"""convert dataframe to SFrame """
train_data = tc.SFrame(train)
test_data = tc.SFrame(test)

In [42]:
train_data

id,productId,purchase_count
1892,66,1
881,216,1
1219,208,1
123,25,1
1695,190,1
1997,148,1
1165,47,1
1376,169,1
760,59,1
977,230,1


In [43]:
test_data

id,productId,purchase_count
911,198,1
548,119,1
458,8,1
132,77,1
2222,240,1
1796,34,1
1783,69,1
1302,50,1
1414,37,1
2419,94,1


In [0]:
### 5.2)- Define a split_data function for splitting data to training and test set
def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [0]:
### 5.3)-Apply for both dummy table and scaled/normalized purchase table

In [0]:
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

In [0]:
#6)-Model

In [0]:
### 6.1)- Parameters to define field names for purchase count as target feature
user_id = 'id'
item_id = 'productId'
users_to_recommend=list(cust_2_rec)
target = 'purchase_count'
n_rec = 10 # number of items to recommend
n_display = 30 # 1st 30 rows to display

In [68]:
popularity_model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 19848 observations with 2350 users and 250 items.

Data prepared in: 0.046237s

19848 observations to process; with 250 unique items.

In [56]:
popularity_recomm = popularity_model.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm.print_rows(n_display)

recommendations finished on 1000/1000 queries. users per second: 80360

+----+-----------+--------------------+------+
| id | productId |       score        | rank |
+----+-----------+--------------------+------+
| 0  |     96    | 1.0886075949367089 |  1   |
| 0  |    201    | 1.088235294117647  |  2   |
| 0  |    112    | 1.0705882352941176 |  3   |
| 0  |     26    | 1.0666666666666667 |  4   |
| 0  |    131    | 1.0609756097560976 |  5   |
| 0  |    153    | 1.0602409638554218 |  6   |
| 0  |     71    | 1.0597014925373134 |  7   |
| 0  |     63    | 1.0588235294117647 |  8   |
| 0  |     35    | 1.0588235294117647 |  9   |
| 0  |    120    | 1.0588235294117647 |  10  |
| 1  |     96    | 1.0886075949367089 |  1   |
| 1  |    201    | 1.088235294117647  |  2   |
| 1  |    125    | 1.0731707317073171 |  3   |
| 1  |    112    | 1.0705882352941176 |  4   |
| 1  |     26    | 1.0666666666666667 |  5   |
| 1  |    131    | 1.0609756097560976 |  6   |
| 1  |    153    | 1.0602409638554218 |  7   |
| 1  |     71    | 1.0597014925373134 |  8   |
| 1  |     63

In [0]:
"""
Through this model, we predicted the recommendation items using scores by popularity. As you can tell for each model results above, the rows show the first 30 records from 1000 users
with 10 recommendations. These 30 records include 3 users and their recommended items, along with score and descending ranks.

"""

In [76]:
# Checking most frequent items  
train.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending=False).head(20)

productId
96     1.088608
201    1.088235
125    1.073171
112    1.070588
26     1.066667
131    1.060976
153    1.060241
71     1.059701
63     1.058824
120    1.058824
207    1.058824
35     1.058824
152    1.056818
7      1.056818
156    1.056180
181    1.054054
88     1.054054
66     1.054054
232    1.053333
18     1.049383
Name: purchase_count, dtype: float64

In [0]:
"""
products 96, 201, 125,112 and 26 are the most popular (best-selling) across customers.


"""

In [0]:
"""



# Define a model
def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model
    
 """

In [0]:
### 6.2)- purchase dummy as target feature
user_id = 'id'
item_id = 'productId'
users_to_recommend=list(cust_2_rec)
target = 'purchase_dummy'
n_rec = 10 # number of items to recommend
n_display = 30 # 1st 30 rows to display

In [71]:
popularity_model_dummy = tc.popularity_recommender.create(train_data_dummy, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 19848 observations with 2350 users and 250 items.

Data prepared in: 0.04132s

19848 observations to process; with 250 unique items.

In [72]:
popularity_recomm_4_dummy = popularity_model_dummy.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm_4_dummy.print_rows(n_display)

recommendations finished on 1000/1000 queries. users per second: 74816.7

+----+-----------+-------+------+
| id | productId | score | rank |
+----+-----------+-------+------+
| 0  |     54    |  1.0  |  1   |
| 0  |    176    |  1.0  |  2   |
| 0  |    214    |  1.0  |  3   |
| 0  |    173    |  1.0  |  4   |
| 0  |    159    |  1.0  |  5   |
| 0  |     71    |  1.0  |  6   |
| 0  |    145    |  1.0  |  7   |
| 0  |     60    |  1.0  |  8   |
| 0  |    189    |  1.0  |  9   |
| 0  |    248    |  1.0  |  10  |
| 1  |     54    |  1.0  |  1   |
| 1  |    176    |  1.0  |  2   |
| 1  |    214    |  1.0  |  3   |
| 1  |    173    |  1.0  |  4   |
| 1  |    159    |  1.0  |  5   |
| 1  |     71    |  1.0  |  6   |
| 1  |    145    |  1.0  |  7   |
| 1  |     60    |  1.0  |  8   |
| 1  |    189    |  1.0  |  9   |
| 1  |    248    |  1.0  |  10  |
| 2  |     54    |  1.0  |  1   |
| 2  |    176    |  1.0  |  2   |
| 2  |    214    |  1.0  |  3   |
| 2  |    173    |  1.0  |  4   |
| 2  |    159    |  1.0  |  5   |
| 2  |     71    |  1.0  |  6   |
| 2  |    145 

In [0]:
### 6.3)- Applying 'scaled_purchase_freq' as target feature on model
user_id = 'id'
item_id = 'productId'
users_to_recommend=list(cust_2_rec)
target = 'scaled_purchase_freq'
n_rec = 10 # number of items to recommend
n_display = 30 # 1st 30 rows to display

In [74]:
popularity_model_scaled = tc.popularity_recommender.create(train_data_norm, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 18024 observations with 2331 users and 227 items.

Data prepared in: 0.03801s

18024 observations to process; with 227 unique items.

In [75]:
popularity_recomm_4_scaled = popularity_model_scaled.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm_4_scaled.print_rows(n_display)

recommendations finished on 1000/1000 queries. users per second: 84495.1

+----+-----------+---------------------+------+
| id | productId |        score        | rank |
+----+-----------+---------------------+------+
| 0  |    207    | 0.08974358974358974 |  1   |
| 0  |    201    | 0.07936507936507936 |  2   |
| 0  |    125    |  0.0759493670886076 |  3   |
| 0  |    152    | 0.07216494845360824 |  4   |
| 0  |     35    | 0.06944444444444445 |  5   |
| 0  |    129    | 0.06756756756756757 |  6   |
| 0  |     26    | 0.06493506493506493 |  7   |
| 0  |    153    | 0.06329113924050633 |  8   |
| 0  |     96    | 0.06329113924050633 |  9   |
| 0  |    150    |        0.0625       |  10  |
| 1  |    207    | 0.08974358974358974 |  1   |
| 1  |    201    | 0.07936507936507936 |  2   |
| 1  |    125    |  0.0759493670886076 |  3   |
| 1  |    152    | 0.07216494845360824 |  4   |
| 1  |     35    | 0.06944444444444445 |  5   |
| 1  |    129    | 0.06756756756756757 |  6   |
| 1  |     26    | 0.06493506493506493 |  7   |
| 1  |    153    | 0.06329113924050633 |

productId
96     1.088608
201    1.088235
125    1.073171
112    1.070588
26     1.066667
131    1.060976
153    1.060241
71     1.059701
63     1.058824
120    1.058824
207    1.058824
35     1.058824
152    1.056818
7      1.056818
156    1.056180
181    1.054054
88     1.054054
66     1.054054
232    1.053333
18     1.049383
Name: purchase_count, dtype: float64